# Modelling

This notebook is used for both training models and using them to generate predictions. Feel free to begin configuring model code!

The (example) models we (may) use are:
1. XGBoost
2. Random Forest Classifier
3. Temporal Fusion Transformer

There are placeholder sections for both Random Forest Classifier and Temporal Fusion Transformer, but these are not meant to be included. They are just placeholders in case we wanbt to include more models. We need at least two models.

## Setup

### Imports

### Data Loading
I have used load/save functions before, but I have not checked if they work properly in this project. Please fix the code if you find errors.

In [ ]:
def load_data(filename, folder="raw"):
    """
    Load the data from the specified folder with appropriate delimiter.
    
    Parameters
    ----------
    filename : str
        The name of the file to load.
    folder : str, optional
        The folder to load the data from. Default is "raw".
    
    Returns
    -------
    pandas.DataFrame or geopandas.GeoDataFrame
        The loaded data.
    """
    # Define delimiters for different file types
    delimiters = {
        'ais_test.csv': ',',
        'ais_sample_submission.csv': ',',
        'default': '|'
    }
    
    BASE_DIR = os.getcwd()
    file_path = os.path.join(BASE_DIR, f"../data/{folder}/{filename}")
    
    try:
        if folder == "maps":
            # Load map data
            if "land" in filename:
                df = gpd.read_file(file_path)
            elif "ocean" in filename:
                df = gpd.read_file(file_path)
            else:
                raise ValueError("Unsupported map file")
        else:
            # Get correct delimiter
            delimiter = delimiters.get(filename, delimiters['default'])
            # Load CSV data
            df = pd.read_csv(file_path, sep=delimiter)
        
        print(f"Data loaded successfully from {file_path}")
        return df
    except Exception as e:
        print(f"An error occurred while loading the file: {e}")

def save_data(df, filename, folder="2_interim"):
    """
    Save the dataframe to a CSV file in the specified folder.
    
    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe to save.
    filename : str
        The name of the file to save.
    folder : str, optional
        The folder to save the data in. Default is "2_interim".
    """
    BASE_DIR = os.getcwd()
    file_path = os.path.join(BASE_DIR, f"../data/{folder}/{filename}.csv")
    
    try:
        df.to_csv(file_path, sep='|', index=False)
        print(f"Data saved successfully to {file_path}")
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")

### Helper Functions

## XGBoost

This section implements the XGboost model. Then the model is used for prediction.

Pros:
- Quick training
- Handles label-encoding of categoricals
- Performant with limited data
- Handles missing values

Cons:
- Requires good-quality training data

Got a tip that XGBoost can be configured to use quantile regression as the objective function. We need to use something like ```reg:quantileerror``` with the parameter ```alpha=0.2```. This way we optimize the model for the 20-percent quantile, such as the task asks for.

### Training

### Prediction

## Random Forest Classifier

### Training

### Prediction

## Temporal Fusion Transformer
If we want to experiment with more models.

Pros:
- Handles irregular timestamps
- Can learn "deeper" trends than xgboost
- Handles missing values
- Can do "target encoding" of categoricals instead of one-hot encoding.

Cons:
- Long training time

### Training

### Prediction